# Create meeting minutes from an Audio file

In [25]:
!pip install -q --upgrade bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.6 MB/s eta 0:00:00


In [ ]:
!pip install datasets soundfile pydub
!apt-get install -y ffmpeg

In [2]:
import os

filepath = "/content/show-train-data-sample.json"

# 1. Verificar si el archivo existe
if not os.path.exists(filepath):
    print("❌ El archivo NO existe.")
else:
    # 2. Verificar el tamaño del archivo
    size = os.path.getsize(filepath)
    print(f"📁 Tamaño del archivo: {size} bytes")

    if size == 0:
        print("⚠️ EL ARCHIVO ESTÁ VACÍO. Por eso falla el json.load.")
    else:
        # 3. Leer los primeros 100 caracteres para ver qué contiene
        with open(filepath, "r", encoding="utf-8") as f:
            content = f.read(100)
            print(f"👀 Contenido inicial: '{content}'")

📁 Tamaño del archivo: 324327 bytes
👀 Contenido inicial: '[
	{"summary":"AS AMENDED a bill for an ordinance amending the Denver Zoning Code to revise parking '


In [9]:
import torch
print(torch.cuda.is_available())

False


In [7]:
import json
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import torch
from IPython.display import Audio

# 1. Cargar JSON
with open("/content/show-train-data-sample.json", "r", encoding="utf-8") as f:
    data = json.load(f)

text = data[0]["summary"]


processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to('cuda')
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to('cuda')

# Create a default speaker embedding
speaker_embedding = torch.randn(1, 512).to('cuda')

# Generate speech
inputs = processor(text=text, return_tensors="pt").to('cuda')
speech = model.generate_speech(inputs["input_ids"], speaker_embedding, vocoder=vocoder)

Audio(speech.cpu().numpy(), rate=16000)

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

AssertionError: Torch not compiled with CUDA enabled

In [19]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [28]:
# Constants
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Llama-3.1-8B-Instruct"

In [13]:
# New capability - connect this Colab to your Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3
# Place the file on your drive in a folder called llms, and call it denver_extract.mp3
from google.colab import drive
drive.mount("/content/drive")
audio_filename = "/content/denver_extract.mp3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Sign in to HuggingFace Hub
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HF_API_KEY')
login(hf_token, add_to_git_credential=True)

# Open the file

audio_file = open(audio_filename, "rb")

# STEP 1: Transcribe Audio

## Option 1: Use Open Source for Transcription - Hugging Face Pipelines

In [17]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-medium.en",
    dtype=torch.float16,
    device='cuda',
    return_timestamps=True
)

result = pipe(audio_filename)
transcription = result["text"]
print(transcription)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda


AssertionError: Torch not compiled with CUDA enabled

## Option 2: Use OpenAI for Transcription

In [20]:
# Sign in to OpenAI using Secrets in Colab

AUDIO_MODEL = "gpt-4o-mini-transcribe"

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

kind of the confluence of this whole idea of a confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let you see the creation of the logo here. Yeah, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United S

In [ ]:
#display(Markdown(open_source_transcription))
print("\n\n")
display(Markdown(transcription))

# STEP 2: Analyze & Report

In [23]:
system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [26]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [29]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
response = tokenizer.decode(outputs[0])

In [ ]:
display(Markdown(response))